In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, TimeDistributed, GlobalAveragePooling1D, Add, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics
import os
import random
import tensorflow as tf

In [2]:
# reset the seeds for reproducibility
def reset_random_seeds():
    n = 1
    os.environ['PYTHONHASHSEED'] = str(n)
    tf.random.set_seed(n)
    np.random.seed(n)
    random.seed(n)

reset_random_seeds()

# import data
df = pd.read_csv('onchain_data_new.csv')
df1 = pd.read_csv('all_data.csv')

df = df[df['timestamp'] >= '2013-03-11'].reset_index(drop=True)

#onchain_data_new
#TA_data
#all_data
#Boruta_data
#Boruta_TA_data
#Boruta_onchain_data

In [3]:
# separate the inputs and target
X = df.drop('timestamp', axis=1)

# create binary classification for price movement. this assigns 1 to y if price will move upward next day.
price = pd.DataFrame()
price['today'] = df1['price-ohlc-usd-c']
price['next day'] = price['today'].shift(-1)
y = price['next day']

# Drop the last row where 'next day' would be NaN after shifting
X = X[:-1]
y = y.dropna()

# separate training data from testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [4]:
# scale the input data
scaler = StandardScaler()

# Reshape X_train and X_test if they are 1D
if X_train.ndim == 1:
    X_train = X_train.to_numpy().reshape(-1, 1)
if X_test.ndim == 1:
    X_test = X_test.to_numpy().reshape(-1, 1)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# reshape the input data for CNN-LSTM (samples, timesteps, features)
def create_sequences(data, timesteps):
    X = []
    for i in range(len(data) - timesteps + 1):
        X.append(data[i:i + timesteps])
    return np.array(X)

timesteps = 5
X_train_reshaped = create_sequences(X_train_scaled, timesteps)
X_test_reshaped = create_sequences(X_test_scaled, timesteps)
y_train = y_train[timesteps - 1:]
y_test = y_test[timesteps - 1:]

In [5]:
# define the LSTNet model
input_layer = Input(shape=(timesteps, X_train_reshaped.shape[2]))

# CNN component
cnn = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
cnn = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn)
cnn = TimeDistributed(Dense(64, activation='relu'))(cnn)
cnn = TimeDistributed(Dense(32, activation='relu'))(cnn)
cnn = TimeDistributed(Dense(1))(cnn)
cnn = GlobalAveragePooling1D()(cnn)

# LSTM component
lstm = LSTM(units=256, return_sequences=True)(input_layer)
lstm = LSTM(units=128, return_sequences=True)(lstm)
lstm = TimeDistributed(Dense(64, activation='relu'))(lstm)
lstm = TimeDistributed(Dense(32, activation='relu'))(lstm)
lstm = TimeDistributed(Dense(1))(lstm)
lstm = GlobalAveragePooling1D()(lstm)

# Autoregression (AR) component
ar = Dense(64, activation='linear')(input_layer)
ar = Dense(32, activation='linear')(ar)
ar = Dense(1, activation='linear')(ar)
ar = GlobalAveragePooling1D()(ar)

# Merge all components
merged = Add()([cnn, lstm, ar])
merged = Dense(64, activation='relu')(merged)
merged = Dropout(0.5)(merged)
merged = Dense(32, activation='relu')(merged)
merged = Dropout(0.5)(merged)
merged = Dense(1, activation='relu')(merged)

# Create the model
model = Model(inputs=input_layer, outputs=merged)

# compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error', metrics=['mean_absolute_error', 'mean_squared_error'])

In [6]:
model.save("lstnet_r_model.h5")

In [6]:
# train with the training dataset
early_stopping = EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=50, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
54/54 [==============================] - 12s 59ms/step - loss: 3441.2993 - mean_absolute_error: 3441.2993 - mean_squared_error: 27327164.0000 - val_loss: 31277.7617 - val_mean_absolute_error: 31277.7617 - val_mean_squared_error: 1310170112.0000
Epoch 2/100
54/54 [==============================] - 1s 25ms/step - loss: 3035.5667 - mean_absolute_error: 3035.5667 - mean_squared_error: 21632392.0000 - val_loss: 18805.8164 - val_mean_absolute_error: 18805.8164 - val_mean_squared_error: 468503680.0000
Epoch 3/100
54/54 [==============================] - 1s 22ms/step - loss: 1263.0974 - mean_absolute_error: 1263.0974 - mean_squared_error: 4252673.0000 - val_loss: 2634.8579 - val_mean_absolute_error: 2634.8577 - val_mean_squared_error: 16692418.0000
Epoch 4/100
54/54 [==============================] - 1s 23ms/step - loss: 888.7354 - mean_absolute_error: 888.7354 - mean_squared_error: 2619014.7500 - val_loss: 3439.0271 - val_mean_absolute_error: 3439.0271 - val_mean_squared_error: 19

In [7]:
# predict with testing dataset
y_pred = model.predict(X_test_reshaped)

24/24 [==============================] - 1s 7ms/step


In [8]:
# evaluate the prediction performance
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

Root Mean Squared Error: 3805.5101629258975
Mean Absolute Error: 3261.7465062621154
R^2 Score: 0.9112864791406927


In [9]:
time = df['timestamp']
time_train, time_test = train_test_split(time, test_size=0.2, shuffle=False)
time_test = time_test[timesteps:]

# Flatten y_pred to be a 1-dimensional array
y_pred_flat = y_pred.flatten()

# Create a DataFrame with columns time_test, y_test, and y_pred
pred_res = pd.DataFrame({'date': time_test, 'actual': y_test.values, 'prediction': y_pred_flat, 'value': price['today'][-748:]})

pred_res.to_csv('pred2/lstnet_uni_data.csv', index=False)